In [1]:
# import required packages
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [2]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [3]:
# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        'data/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 17010 images belonging to 3 classes.


In [4]:
# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')


Found 4254 images belonging to 3 classes.


In [5]:
# create model structure
emotion_model = Sequential()

In [6]:
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

In [7]:
# emotion_model.add(Flatten())
# emotion_model.add(Dense(1024, activation='relu'))
# emotion_model.add(Dropout(0.5))
# emotion_model.add(Dense(7, activation='softmax'))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(3, activation='softmax'))

In [8]:
cv2.ocl.setUseOpenCL(False)

In [9]:
emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

C:\Users\ankit\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [10]:
# Train the neural network/model
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=17010 // 64,  #448
        epochs=25,
        validation_data=validation_generator,
        validation_steps=4254 // 64)  #112

<ipython-input-10-47bda955c933>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


Epoch 1/25
265/265 [==============================] - 294s 1s/step - loss: 1.0756 - accuracy: 0.4251 - val_loss: 1.0644 - val_accuracy: 0.4384
Epoch 2/25
265/265 [==============================] - 158s 595ms/step - loss: 0.9758 - accuracy: 0.5220 - val_loss: 0.9057 - val_accuracy: 0.5930
Epoch 3/25
265/265 [==============================] - 291s 1s/step - loss: 0.8834 - accuracy: 0.5928 - val_loss: 0.8288 - val_accuracy: 0.6259
Epoch 4/25
265/265 [==============================] - 224s 846ms/step - loss: 0.8234 - accuracy: 0.6213 - val_loss: 0.7793 - val_accuracy: 0.6463
Epoch 5/25
265/265 [==============================] - 230s 869ms/step - loss: 0.7788 - accuracy: 0.6413 - val_loss: 0.7531 - val_accuracy: 0.6600
Epoch 6/25
265/265 [==============================] - 255s 965ms/step - loss: 0.7493 - accuracy: 0.6622 - val_loss: 0.7372 - val_accuracy: 0.6605
Epoch 7/25
265/265 [==============================] - 174s 656ms/step - loss: 0.7230 - accuracy: 0.6757 - val_loss: 0.7095 - val_a

In [11]:
# save model structure in json file
model_json = emotion_model.to_json()
with open("new_emotion_model.json", "w") as json_file:
    json_file.write(model_json)

In [12]:
# save trained model weight in .h5 file
emotion_model.save_weights('new_emotion_model.h5')